In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets,transforms

In [3]:
transform = transforms.ToTensor()

train_set = datasets.MNIST('../data/MNIST_data/',download=False,train=True,transform=transform)
trainloader = torch.utils.data.DataLoader(train_set,batch_size=64,shuffle=True)

test_set = datasets.MNIST('../data/MNIST_data/',download=False,train=False,transform=transform)
testloader = torch.utils.data.DataLoader(test_set,batch_size=64,shuffle=True)

In [30]:
class Network(nn.Module):
    
    def __init__(self):
        super().__init__();
        
        self.conv=nn.Conv2d(1,5,kernel_size=(3,3),stride=(1,1),padding=1)
        self.maxpool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1=nn.Linear(980,512)
        self.fc2=nn.Linear(512,512)
        self.fc3=nn.Linear(512,10)
        
    def forward(self,x):

        x=self.conv(x)
        x=self.maxpool(x)
        x=self.dropout(x)
        x=F.relu(x)
        x=x.view(x.shape[0],-1)
        
        x=self.fc1(x)
        x=self.dropout(x)
        x=F.relu(x)
        x=self.fc2(x)
        x=self.dropout(x)
        x=F.relu(x)
        x=self.fc3(x)
        x=F.log_softmax(x,dim=1)
        
        return x

In [31]:
model=Network()

In [32]:
print(model)

Network(
  (conv): Conv2d(1, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.2)
  (fc1): Linear(in_features=980, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
)


In [23]:
def train(Model,trainloader,testloader,criterion,optimizer,epochs):
    test_losses,train_losses=[],[]
    for e in range(epochs):
        running_loss=0;

        Model.train();
        for images,labels in trainloader:
#             images_t = images.view(images.shape[0],-1);
            optimizer.zero_grad();

            logits=Model.forward(images);
            loss_t=criterion(logits,labels);
            loss_t.backward();
            optimizer.step();

            running_loss+=loss_t;

        else:
            test_loss,accuracy=validation(Model,testloader,criterion);

            print("Epoch: {}/{}.. ".format(e+1, epochs),
                      "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
                      "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
                      "Test Accuracy: {:.3f}".format(accuracy))

        test_losses.append(test_loss/len(testloader))
        train_losses.append(running_loss/len(trainloader))

    return train_losses,test_losses,accuracy

def validation(Model,testloader,criterion):
    test_loss=0;
    accuracy=0;

    Model.eval();
    images_num=0;
    with torch.no_grad():
        for images,labels in testloader:
            images_num+=images.shape[0];
#             images_t=images.view(images.shape[0],-1);
            logits=Model.forward(images);
            loss_t=criterion(logits,labels)
            test_loss+=loss_t;

            _,pred_labels=torch.topk(logits,1,dim=1)
            equality=(labels==pred_labels.view(*labels.shape))

            accuracy += torch.sum(equality)

    overall_acc=accuracy.float()/images_num;
    return test_loss,overall_acc


In [33]:
criterion = nn.NLLLoss()
optimizer=torch.optim.Adam(model.parameters())

In [34]:
train_loss,test_loss,accuracy=train(model,trainloader,testloader,
                                             criterion,optimizer,5)

Epoch: 1/5..  Training Loss: 0.306..  Test Loss: 0.101..  Test Accuracy: 0.967
Epoch: 2/5..  Training Loss: 0.116..  Test Loss: 0.071..  Test Accuracy: 0.978
Epoch: 3/5..  Training Loss: 0.088..  Test Loss: 0.061..  Test Accuracy: 0.980
Epoch: 4/5..  Training Loss: 0.073..  Test Loss: 0.051..  Test Accuracy: 0.983
Epoch: 5/5..  Training Loss: 0.061..  Test Loss: 0.048..  Test Accuracy: 0.984
